In [1]:
import os
print(os.getcwd())
# os.chdir("..")

from src.constants import *
from src.model.my_model import MyModel
from src.model.baselines import UNet, SWINUNETR
import torch
from src.utils.metrics import *
from torch.cuda.amp import GradScaler, autocast
from monai.networks import one_hot
from src.data.data_loaders import get_loaders
from tqdm import tqdm


/home/imreb/AAAsegmentor/notebooks


<class 'monai.transforms.utility.dictionary.AddChanneld'>: Class `AddChanneld` has been deprecated since version 0.8. It will be removed in version 1.3. please use MetaTensor data type and monai.transforms.EnsureChannelFirstd instead with `channel_dim='no_channel'`.
monai.transforms.utility.dictionary EnsureChannelFirstd.__init__:meta_keys: Argument `meta_keys` has been deprecated since version 0.9. not needed if image is type `MetaTensor`.
<class 'monai.transforms.utility.dictionary.AsChannelFirstd'>: Class `AsChannelFirstd` has been deprecated since version 0.8. It will be removed in version 1.3. please use MetaTensor data type and monai.transforms.EnsureChannelFirstd instead.


In [2]:
device = "cuda"

In [ ]:
# s=128
# example = torch.rand(size=(1, 1, 256, 256, 64)).to(device)
# mask = torch.rand(size=(4, 3, 256, 256, 64)).to(device)

# example.dtype

In [ ]:
train_loader, test_loader = get_loaders()


In [3]:
model = MyModel(in_channels=1,
                mid_channels=4,
                out_channels=3,
                patch_size=(4,4,4),
                embed_dim=256,
                img_size=(256, 256, 64)).to(device)

In [6]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)
losses = []

for d in tqdm(train_loader):

    img = d['img'].get_array(output_type=torch.Tensor).to(device)
    mask = d['mask'].get_array(output_type=torch.Tensor).to(device)

    optimizer.zero_grad()
    outputs = model(img)
    outputs = torch.softmax(outputs, dim=1)
    mask = one_hot(mask, num_classes=3)
    loss = LOSS(outputs, mask)
    loss.backward()
    optimizer.step()

    losses.append(loss.detach().cpu().item())
    

100%|██████████| 6/6 [00:08<00:00,  1.39s/it]


In [5]:
torch.set_float32_matmul_precision("medium")

In [6]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)
scaler = GradScaler()
mask = torch.rand(size=(2, 3, 256, 256, 64)).to(device)


for i in tqdm(range(19*4)):
    example = torch.rand(size=(2, 1, 256, 256, 64)).to(device)
    out = model(example)
    out = torch.softmax(out, dim=1)
    loss = LOSS(out, mask)

    loss.backward()
    optimizer.step()

100%|██████████| 76/76 [02:42<00:00,  2.14s/it]


In [10]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)
scaler = GradScaler()
mask = torch.rand(size=(3, 3, 256, 256, 64)).to(device)


for i in tqdm(range(19*4)):
    example = torch.rand(size=(3, 1, 256, 256, 64)).to(device)
    with autocast():
        optimizer.zero_grad()

        out = model(example)
        out = torch.softmax(out, dim=1)
        loss = LOSS(out, mask)

    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

100%|██████████| 76/76 [02:14<00:00,  1.76s/it]


In [21]:
img = next(iter(train_loader))['img'].get_array(output_type=torch.Tensor).to(device)